# Gerenate using ChatGPT
We will generate 10 texts for each abstract across two temperatures. 

In [10]:
%load_ext autoreload
%autoreload 2


In [11]:
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  

# dan jurafsky's string2string library
from string2string.similarity import JaroSimilarity


from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from prompting.prompts import DataTemplates

In [68]:
df = pd.read_csv("../abstracts_final.csv")

In [69]:
template = ChatPromptTemplate.from_messages(DataTemplates().medicine_abstract_prompt())

In [72]:
def generate_row(text: str, temperature) -> str:
    llm = ChatOpenAI(temperature=temperature)
    chat = LLMChain(llm=llm, prompt=template)
    output = chat.run({"text": text})
    return output

In [108]:
temperatures = [0.7, 0.9]

In [88]:
import concurrent.futures

def generate_wrapper(row, t):
    text = row["texts"]
    return {"temperature": t, "text": generate_row(text, t)}

generations = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for j in range(10):
        for t in temperatures:
            # Prepare arguments for each row
            args = [(row, t) for index, row in df.iterrows()]b

            # Execute the generate_wrapper function with each row in a separate thread
            results = list(executor.map(lambda params: generate_wrapper(*params), args))
            
            # Add results to generations list
            generations.extend(results)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d675412aa67423dc58f2d9ee3638cad5 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fe226e1ef25567f2e19d9578176f234f in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can r

In [76]:
# generations = []
# while j < 10:
#     for t in temperatures:
#         for i, row in df.iterrows():
#             text = row["texts"]
#             generations.append({"temperature": t, "text": generate_row(text, t)})
#     j += 1

In [122]:
df2 = pd.DataFrame(generations).reset_index()
df2["index"] = df2["index"].apply(lambda x: x % 16)

In [114]:
id_labels = pd.read_csv("../data/qids.csv")

In [120]:
id_labels.reset_index(inplace=True)

In [125]:
df2 = df2.merge(id_labels[["index", "HITId"]], on = "index")

In [111]:
# df[temperatures] = df2.pivot(index="index", columns="temperature", values="text")

In [128]:
df2.to_csv("../data/generated_outputs.csv", index=False)

In [12]:
gen = pd.read_csv("../data/generated_outputs.csv")

In [ ]:
results = ["ai", "ai", "human", "human", "human", "AI", "AI", "AI", "AI", "human"]


